In [1]:
# the following script contains portions of code used by Digital Earth Australia in their tutorials:
# https://docs.dea.ga.gov.au/notebooks/Scientific_workflows/TSmask/TSmask.html, 
# under Apache License, Version 2.0 (https://www.apache.org/licenses/LICENSE-2.0)
# DEA data is under Creative Commons by Attribution 4.0 license 
# (https://creativecommons.org/licenses/by/4.0/)

In [9]:
%matplotlib inline
import sys
import matplotlib.pyplot as plt
import datacube
import xarray as xr
from datacube.utils.masking import make_mask
from datacube.drivers.netcdf import write_dataset_to_netcdf
import numpy as np
from numpy import inf
import pandas as pd
#sys.path

## Download and load the Random Forest model.

In [10]:
!pip3 install pickle5

In [11]:
import pickle5 as pickle
with open('rf_fmc.pickle', 'rb') as handle:
    rf = pickle.load(handle)
    print(rf)

RandomForestRegressor(criterion='mse', max_depth=25, max_features='auto',
                      n_estimators=25, n_jobs=8)


/env/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.23.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/env/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.23.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Load DEA data for region of interest

In [16]:
dc = datacube.Datacube(app='fmc')

In [13]:
# Set some configurations for displaying tables nicely
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", None)

products = dc.list_products()
products

,name,description,license,default_crs,default_resolution
name,,,,,
aster_aloh_group_composition,aster_aloh_group_composition,ASTER,None,None,None
aster_aloh_group_content,aster_aloh_group_content,ASTER,None,None,None
aster_false_colour,aster_false_colour,ASTER,None,None,None
aster_feoh_group_content,aster_feoh_group_content,ASTER,None,None,None
aster_ferric_oxide_composition,aster_ferric_oxide_composition,ASTER,None,None,None
aster_ferric_oxide_content,aster_ferric_oxide_content,ASTER,None,None,None
aster_ferrous_iron_content_in_mgoh,aster_ferrous_iron_content_in_mgoh,ASTER,None,None,None
aster_ferrous_iron_index,aster_ferrous_iron_index,ASTER,None,None,None
aster_green_vegetation,aster_green_vegetation,ASTER,None,None,None


In [15]:
#check variables names
product = "ga_s2am_ard_3"
measurements = dc.list_measurements()
measurements.loc[product]

,name,dtype,units,nodata,aliases,flags_definition
measurement,,,,,,
nbart_coastal_aerosol,nbart_coastal_aerosol,int16,1,-999,"[nbart_band01, coastal_aerosol]",NaN
nbart_blue,nbart_blue,int16,1,-999,"[nbart_band02, blue]",NaN
nbart_green,nbart_green,int16,1,-999,"[nbart_band03, green]",NaN
nbart_red,nbart_red,int16,1,-999,"[nbart_band04, red]",NaN
nbart_red_edge_1,nbart_red_edge_1,int16,1,-999,"[nbart_band05, red_edge_1]",NaN
nbart_red_edge_2,nbart_red_edge_2,int16,1,-999,"[nbart_band06, red_edge_2]",NaN
nbart_red_edge_3,nbart_red_edge_3,int16,1,-999,"[nbart_band07, red_edge_3]",NaN
nbart_nir_1,nbart_nir_1,int16,1,-999,"[nbart_band08, nir_1, nbart_common_nir]",NaN
nbart_nir_2,nbart_nir_2,int16,1,-999,"[nbart_band8a, nir_2]",NaN


In [7]:
#check variables names
#product = "ga_s2am_ard_3"
#measurements = dc.list_measurements()
#measurements.loc[product]

In [21]:
# load sentinel sentinel-2a, and sentinel-2b  and merge them
query = {
        'y': (-34.900, -35.500),
        'x': (148.900, 149.500),
        'crs': 'EPSG:4326',
        'output_crs': 'EPSG:4326',
        'resolution': (-0.0002, 0.0002),  #some bands are 10m resolution, but others are 20m
        'time': ('2023-02-17', '2023-02-21'),
        'measurements':["oa_fmask",
                         "nbart_blue","nbart_green","nbart_red",
                         "nbart_red_edge_1","nbart_red_edge_2","nbart_red_edge_3",
                         "nbart_nir_1","nbart_nir_2",
                         "nbart_swir_2","nbart_swir_3"], }



In [22]:
s2a_ds = dc.load(product='ga_s2am_ard_3', group_by='solar_day', **query)

In [23]:
s2b_ds = dc.load(product='ga_s2bm_ard_3', group_by='solar_day', **query)

/env/lib/python3.8/site-packages/datacube/utils/geometry/_base.py:626: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type in ['Point', 'MultiPoint']:
/env/lib/python3.8/site-packages/datacube/utils/geometry/_base.py:629: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type in ['GeometryCollection', 'MultiPolygon', 'MultiLineString']:
/env/lib/python3.8/site-packages/datacube/utils/geometry/_base.py:632: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type in ['LineString', 'LinearRing']:
/env/lib/python3.8/site-packages/datacube/utils/geometry/_base.py:635: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the

In [24]:
#ds_nomask = xr.concat([s2a_ds, s2b_ds], dim='time').sortby('time')

In [25]:
ds_nomask = s2b_ds.sortby('time')

In [26]:
print(ds_nomask)
print(ds_nomask.time.data)

<xarray.Dataset>
Dimensions:           (time: 2, latitude: 3001, longitude: 3000)
Coordinates:
  * time              (time) datetime64[ns] 2023-02-18T00:06:26.322011 2023-0...
  * latitude          (latitude) float64 -34.9 -34.9 -34.9 ... -35.5 -35.5 -35.5
  * longitude         (longitude) float64 148.9 148.9 148.9 ... 149.5 149.5
    spatial_ref       int32 4326
Data variables:
    oa_fmask          (time, latitude, longitude) uint8 1 1 1 1 1 ... 0 0 0 0 0
    nbart_blue        (time, latitude, longitude) int16 711 749 ... -999 -999
    nbart_green       (time, latitude, longitude) int16 1047 1062 ... -999 -999
    nbart_red         (time, latitude, longitude) int16 1395 1479 ... -999 -999
    nbart_red_edge_1  (time, latitude, longitude) int16 1579 1576 ... -999 -999
    nbart_red_edge_2  (time, latitude, longitude) int16 2002 1974 ... -999 -999
    nbart_red_edge_3  (time, latitude, longitude) int16 2206 2257 ... -999 -999
    nbart_nir_1       (time, latitude, longitude) int16 2434

## Add NDVI and NDII normalised indices to the dataset

In [27]:
ds_nomask['ndvi']=((ds_nomask.nbart_nir_1-ds_nomask.nbart_red)/(ds_nomask.nbart_nir_1+ds_nomask.nbart_red))
ds_nomask['ndii']=((ds_nomask.nbart_nir_1-ds_nomask.nbart_swir_2)/(ds_nomask.nbart_nir_1+ds_nomask.nbart_swir_2))

## Save dataset as netCDF

In [28]:
write_dataset_to_netcdf(ds_nomask, 's2_reflectance_no_mask_northcbr.nc')

## Read fmask flags

In [14]:
# print sentinel 2 fmask flags (code from https://docs.dea.ga.gov.au/notebooks/Scientific_workflows/TSmask/TSmask.html,Apache License, Version 2.0 https://www.apache.org/licenses/LICENSE-2.0)
ds_nomask.fmask.flags_definition

{'fmask': {'bits': [0, 1, 2, 3, 4, 5, 6, 7],
  'values': {'0': 'nodata',
   '1': 'valid',
   '2': 'cloud',
   '3': 'shadow',
   '4': 'snow',
   '5': 'water'},
  'description': 'Fmask'}}

## Visualise RGB images

In [15]:
ds = xr.open_dataset('s2_reflectance_no_mask.nc')

In [16]:
#for i in range(ds.time.data.size):
#    ds_ = ds.isel(time=i)
#    ds_[['nbart_red', 'nbart_green', 'nbart_blue']].to_array().plot.imshow(robust=True, figsize=(8,8))

## Open reflectance dataset

In [29]:
ds = xr.open_dataset('s2_reflectance_no_mask_northcbr.nc')
ds

<xarray.Dataset>
Dimensions:           (time: 2, latitude: 3001, longitude: 3000)
Coordinates:
  * time              (time) datetime64[ns] 2023-02-18T00:06:26 2023-02-21T00...
  * latitude          (latitude) float64 -34.9 -34.9 -34.9 ... -35.5 -35.5 -35.5
  * longitude         (longitude) float64 148.9 148.9 148.9 ... 149.5 149.5
    spatial_ref       int32 ...
Data variables: (12/13)
    oa_fmask          (time, latitude, longitude) float32 ...
    nbart_blue        (time, latitude, longitude) float32 ...
    nbart_green       (time, latitude, longitude) float32 ...
    nbart_red         (time, latitude, longitude) float32 ...
    nbart_red_edge_1  (time, latitude, longitude) float32 ...
    nbart_red_edge_2  (time, latitude, longitude) float32 ...
    ...                ...
    nbart_nir_1       (time, latitude, longitude) float32 ...
    nbart_nir_2       (time, latitude, longitude) float32 ...
    nbart_swir_2      (time, latitude, longitude) float32 ...
    nbart_swir_3      (time, latitude, longitude) float32 ...
    ndvi              (time, latitude, longitude) float64 ...
    ndii              (time, latitude, longitude) float64 ...
Attributes:
    date_created:           2023-03-06T11:28:45.254780
    Conventions:            CF-1.6, ACDD-1.3
    history:                NetCDF-CF file created by datacube version '1.8.1...
    geospatial_bounds:      POLYGON ((148.9 -34.8998, 148.9 -35.5000000000000...
    geospatial_bounds_crs:  EPSG:4326
    geospatial_lat_min:     -35.50000000000001
    geospatial_lat_max:     -34.8998
    geospatial_lat_units:   degrees_north
    geospatial_lon_min:     148.9
    geospatial_lon_max:     149.5
    geospatial_lon_units:   degrees_east

In [18]:
ds.time

<xarray.DataArray 'time' (time: 4)>
array(['2022-04-01T00:46:48.000000000', '2022-04-04T00:56:42.000000000',
       '2022-04-06T00:46:42.000000000', '2022-04-09T00:56:37.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2022-04-01T00:46:48 ... 2022-04-09T00:...
    spatial_ref  int32 -2147483647
Attributes:
    standard_name:  time
    long_name:      Time, unix time-stamp
    axis:           T

## Option A: Compute LFMC from 3D array

In [ ]:
# Stack and reshape dataset to be compatible with the RF input
refl = ds[['ndvi','ndii','nbart_red','nbart_green','nbart_blue','nbart_nir_1','nbart_nir_2','nbart_swir_2','nbart_swir_3']].to_array().values

refl_rf = refl.reshape((9,-1)).swapaxes(0,1)

# Convert infinite values to nans
refl_rf = np.nan_to_num(refl_rf, copy=False, nan=np.nan, posinf=np.nan, neginf=np.nan) 

nan_mask = np.isnan(refl_rf) # nan values are not accepted by the RF model. So first they will be replaced with 0 (below), so that the model can run. Then this mask will be applied to restore the nan in the LFMC map
nan_mask_1d = np.sum(np.where(nan_mask,1,0), axis=1) # if 0, there were no nan values in any of the reflectance bands, if a pixel is > 0 then at least one of the bands in that position was nan

refl_rf = np.where(nan_mask, 0, refl_rf)

# Estimate FMC values using RF model and previous reflectance input
rf_lfmc = rf.predict(refl_rf)

rf_lfmc =  np.where(nan_mask_1d>0, np.nan, rf_lfmc) #if a pixel is > 0 then at least one of the bands in that position was nan, or -inf/inf 

lfmc_3d = rf_lfmc.reshape(refl.shape[1:])

# Apply cloud, shadow, snow, water mask
lfmc_3d = np.where(ds.oa_fmask.data!=1, np.nan, lfmc_3d) #1:valid, the mask has value 1 if nodata, cloud shadow, snow or water, 0 if valid



## (Option A) Create new netCDF for final output

In [ ]:
import netCDF4
import pandas as pd

with netCDF4.Dataset('s2_lfmc_option_A_northcbr.nc', 'w', format='NETCDF4_CLASSIC') as new_ds:
    
    setattr(new_ds, 'crs', 'EPSG:4326')
    setattr(new_ds, 'fmask_legend', '0:nodata, 1:valid, 2:cloud, 3:shadow, 4:snow, 5:water')
    
    t_dim = new_ds.createDimension('time', ds.time.size)
    x_dim = new_ds.createDimension('longitude', ds.longitude.size)
    y_dim = new_ds.createDimension('latitude', ds.latitude.size)
    
    var = new_ds.createVariable("time", "f8", ("time",))
    var.units = "seconds since 1970-01-01 00:00:00.0"
    var.calendar = "standard"
    var.long_name = "Time, unix time-stamp"
    var.standard_name = "time"
    var[:] = [netCDF4.date2num([pd.to_datetime(i)], units="seconds since 1970-01-01 00:00:00.0", calendar="standard") for i in ds.time.data]

    var = new_ds.createVariable("longitude", "f8", ("longitude",))
    var.units = "degrees"
    var.long_name = "longitude"
    var.standard_name = "longitude"
    var[:] = ds.longitude.data

    var = new_ds.createVariable("latitude", "f8", ("latitude",))
    var.units = "degrees"
    var.long_name = "latitude"
    var.standard_name = "latitude"
    var[:] = ds.latitude.data
    
    # if the LFMC maps are not already masked, it is suggested to include fmask and RGB bands in the netCDF file, in order to be able to mask and visually check the maps later
    
    #var = new_ds.createVariable("fmask", 'i4', ("time", "latitude", "longitude"))
    #var.units = ''
    #var.long_name = "Quality mask"
    #var[:] = ds.fmask.data
    
    #var = new_ds.createVariable("nbart_red", 'f8', ("time", "latitude", "longitude"))
    #var.units = ''
    #var.long_name = "Red band"
    #var[:] = ds.nbart_red.data
    
    #var = new_ds.createVariable("nbart_green", 'f8', ("time", "latitude", "longitude"))
    #var.units = ''
    #var.long_name = "Green band"
    #var[:] = ds.nbart_green.data
    
    #var = new_ds.createVariable("nbart_blue", 'f8', ("time", "latitude", "longitude"))
    #var.units = ''
    #var.long_name = "Blue band"
    #var[:] = ds.nbart_blue.data

    var = new_ds.createVariable("lfmc", 'f4', ("time", "latitude", "longitude"), fill_value=-9999.9)
    var.units = '%'
    var.long_name = "Live Fuel Moisture Content estimated with Random Forest model"
    var[:] = lfmc_3d


## Option B: Compute LFMC with loop over dates

In [23]:
list_lfmc_arrays = list()


for d in ds.time.data:
    print('Current date: ', d, '... Ends at: ', ds.time.data[-1])
    
    ds_2d = ds.sel(time=d)
    
    # Stack and reshape dataset to be compatible with the RF input
    refl = ds_2d[['ndvi','ndii','nbart_red','nbart_green','nbart_blue','nbart_nir_1','nbart_nir_2','nbart_swir_2','nbart_swir_3']].to_array().values
    
    # Convert infinite values to nans
    refl = np.nan_to_num(refl, copy=False, nan= np.nan, posinf=np.nan, neginf=np.nan)

    # Check and mask values not accepted by RF model
    nan_mask = np.isnan(refl) # nan values are not accepted by the RF model. So first they will be replaced with 0 (below), so that the model can run. Then this mask will be applied to restore the nan in the LFMC map
    nan_mask_2d = np.sum(np.where(nan_mask,1,0), axis=0) # if 0, there were no nan values in any of the reflectance bands, if a pixel is > 0 then at least one of the bands in that position was nan
    
    refl = np.where(nan_mask, 0, refl) # replace nan values with 0 so that the model can run
    
    refl_rf = refl.reshape((9,-1)).swapaxes(0,1)
    
    # Estimate FMC values using RF model and previous reflectance input
    rf_lfmc = rf.predict(refl_rf)
    lfmc = rf_lfmc.reshape(refl.shape[1:])
    
    lfmc = np.where(nan_mask_2d>0, np.nan, lfmc) #if a pixel is > 0 then at least one of the bands in that position was nan, or -inf/inf 
    
    # Apply cloud, shadow, snow, water mask
    lfmc = np.where(ds_2d.oa_fmask.data!=1, np.nan, lfmc)  #1:valid, the mask has value 1 if nodata, cloud shadow, snow or water, 0 if valid
    
    list_lfmc_arrays.append(lfmc)
  


Current date:  2022-04-01T00:46:48.000000000 ... Ends at:  2022-04-09T00:56:37.000000000
Current date:  2022-04-04T00:56:42.000000000 ... Ends at:  2022-04-09T00:56:37.000000000
Current date:  2022-04-06T00:46:42.000000000 ... Ends at:  2022-04-09T00:56:37.000000000
Current date:  2022-04-09T00:56:37.000000000 ... Ends at:  2022-04-09T00:56:37.000000000


## (Option B) Create new netCDF for final output

In [24]:
import netCDF4
import pandas as pd

with netCDF4.Dataset('s2_lfmc_option_B.nc', 'w', format='NETCDF4_CLASSIC') as new_ds:
    
    setattr(new_ds, 'crs', 'EPSG:4326')
    setattr(new_ds, 'fmask_legend', '0:nodata, 1:valid, 2:cloud, 3:shadow, 4:snow, 5:water')
    
    t_dim = new_ds.createDimension('time', ds.time.size)
    x_dim = new_ds.createDimension('longitude', ds.longitude.size)
    y_dim = new_ds.createDimension('latitude', ds.latitude.size)
    
    var = new_ds.createVariable("time", "f8", ("time",))
    var.units = "seconds since 1970-01-01 00:00:00.0"
    var.calendar = "standard"
    var.long_name = "Time, unix time-stamp"
    var.standard_name = "time"
    var[:] = [netCDF4.date2num([pd.to_datetime(i)], units="seconds since 1970-01-01 00:00:00.0", calendar="standard") for i in ds.time.data]

    var = new_ds.createVariable("longitude", "f8", ("longitude",))
    var.units = "degrees"
    var.long_name = "longitude"
    var.standard_name = "longitude"
    var[:] = ds.longitude.data

    var = new_ds.createVariable("latitude", "f8", ("latitude",))
    var.units = "degrees"
    var.long_name = "latitude"
    var.standard_name = "latitude"
    var[:] = ds.latitude.data
    
    # if the LFMC maps are not already masked, it is suggested to include fmask and RGB bands in the netCDF file, in order to be able to mask and visually check the maps later
    
    #var = new_ds.createVariable("fmask", 'i4', ("time", "latitude", "longitude"))
    #var.units = ''
    #var.long_name = "Quality mask"
    #var[:] = ds.fmask.data
    
    #var = new_ds.createVariable("nbart_red", 'f8', ("time", "latitude", "longitude"))
    #var.units = ''
    #var.long_name = "Red band"
    #var[:] = ds.nbart_red.data
    
    #var = new_ds.createVariable("nbart_green", 'f8', ("time", "latitude", "longitude"))
    #var.units = ''
    #var.long_name = "Green band"
    #var[:] = ds.nbart_green.data
    
    #var = new_ds.createVariable("nbart_blue", 'f8', ("time", "latitude", "longitude"))
    #var.units = ''
    #var.long_name = "Blue band"
    #var[:] = ds.nbart_blue.data

    var = new_ds.createVariable("lfmc", 'f4', ("time", "latitude", "longitude"), fill_value=-9999.9)
    var.units = '%'
    var.long_name = "Live Fuel Moisture Content estimated with Random Forest model"
    for i in range(ds.time.size):
        var[i,:,:] = list_lfmc_arrays[i]


## Create colormap consistent with the current Australian Fuel Monitoring System

In [2]:
lfmc_final = xr.open_dataset('s2_lfmc.nc') ### change file name as needed
lfmc_final

<xarray.Dataset>
Dimensions:      (time: 22, longitude: 2000, latitude: 2500)
Coordinates:
  * time         (time) datetime64[ns] 2022-04-01T00:46:48 ... 2022-05-24T00:...
  * longitude    (longitude) float64 138.5 138.5 138.5 ... 138.9 138.9 138.9
  * latitude     (latitude) float64 -34.6 -34.6 -34.6 ... -35.1 -35.1 -35.1
Data variables:
    fmask        (time, latitude, longitude) int32 ...
    nbart_red    (time, latitude, longitude) float64 ...
    nbart_green  (time, latitude, longitude) float64 ...
    nbart_blue   (time, latitude, longitude) float64 ...
    lfmc         (time, latitude, longitude) float32 ...
Attributes:
    crs:           EPSG:4326
    fmask_legend:  0:nodata, 1:valid, 2:cloud, 3:shadow, 4:snow, 5:water

In [1]:
#%matplotlib inline
#import matplotlib.pyplot as plt
#from matplotlib.colors import LinearSegmentedColormap
#import numpy as np


#for i in range(ds.time.data.size):
#    fmc_array = lfmc_final.lfmc.data[i,...]

#    colors = [(0.87, 0, 0), (1, 1, 0.73), (0.165, 0.615, 0.957)]  # R -> G -> B
#    cmap = LinearSegmentedColormap.from_list('fmc', colors, N=256)
#    plt.figure(figsize=(10,10))
#    plt.imshow(np.clip(fmc_array, 0, 136), cmap=cmap)
#    plt.colorbar()


In [2]:
#for i in range(lfmc_final.time.data.size):
#    ds_ = lfmc_final.isel(time=i)
#    ds_[['nbart_red', 'nbart_green', 'nbart_blue']].to_array().plot.imshow(robust=True, figsize=(8,8))


In [3]:
#lfmc_final_masked = xr.open_dataset('s2_lfmc_noclouds_noshadows.nc') ### change file name as needed

#%matplotlib inline
#import matplotlib.pyplot as plt
#from matplotlib.colors import LinearSegmentedColormap
#import numpy as np


#for i in range(lfmc_final_masked.time.data.size):
#    fmc_array = lfmc_final_masked.lfmc.data[i,...]

#    colors = [(0.87, 0, 0), (1, 1, 0.73), (0.165, 0.615, 0.957)]  # R -> G -> B
#    cmap = LinearSegmentedColormap.from_list('fmc', colors, N=256)
#    plt.figure(figsize=(10,10))
#    plt.imshow(np.clip(fmc_array, 0, 136), cmap=cmap)
#    plt.colorbar()
